In [1]:
from riverine import *

# Concepts

Mixes ...

## Components

Components ...

The base `Component` class is meant to usable generically.  For example, we might have a solution of MgCl₂ that we'd like to use to make a Mg-added buffer.

In [2]:
mg = Component("MgCl₂", "1 M")
print(mg)

Component(name='MgCl₂', concentration=<Quantity(1, 'molar')>, plate='', well=None)


Like many components in mixes, the concentration is easiest to enter as a string.  mixes uses the [pint](https://github.com/hgrecco/pint) library to handle units, and Python's [decimal](FIXME) library to avoid imprecision in calculations.  It does this as transparently as possible: you can enter most values with units as either a string, or a pint Quantity, and they will be converted correctly.  You can use `Q_` as a shorthand to create a Quantity from a string, or a number for the value and string for the units.  The input should be quite flexible, for example:

In [3]:
Q_("5 µM") == Q_(5, "µM") == Q_("5 micromolar") == Q_("5 pmol / microliter")

True

In addition to having a name and a concentration, a component can have a location (currently using the `plate` property), and, if the location is a plate name, can also be given a `well`.

## Actions and Mixes

mixes combines Components into Mixes through Actions.  Actions specify what we'd like to do with a component, or a list of components, when we add them to a mix.  For example, we might want to make a buffer stock with 125 mM of MgCl₂ in it, in which case we could use the `FixedConcentration` action, which adds a single component at a fixed target concentration:

In [4]:
add_mg = FixedConcentration(mg, "125 mM")

A `Mix`, then, is a list of these actions, together with some overall properties, like a name or 

In [5]:
mg_buffer = Mix([add_mg], "10× Mg", fixed_total_volume="1 mL")

In [6]:
mg_buffer

Table: Mix: 10× Mg, Conc: 125.00 mM, Total Vol: 1.00 ml

| Component   | [Src]   | [Dest]      | #   | Ea Tx Vol   | Tot Tx Vol   | Location   | Note   |
|:------------|:--------|:------------|:----|:------------|:-------------|:-----------|:-------|
| MgCl₂       | 1.00 M  | 125.00 mM   |     | 125.00 µl   | 125.00 µl    |            |        |
| Buffer      |         |             |     | 875.00 µl   | 875.00 µl    |            |        |
| *Total:*    |         | *125.00 mM* | *2* |             | *1.00 ml*    |            |        |

As we will see later, a `Mix` itself can also be a component in other mixes.

# Strands and References

A `Strand` is a type of component that also keeps track of a sequence:

In [7]:
Strand("S1", concentration="100 µM", sequence="AGAAT")

Strand(name='S1', concentration=<Quantity(100, 'micromolar')>, plate='', well=None, sequence='AGAAT')

Specifying all properties of every component in code would be time consuming and error prone.  Instead, we can specify the components without all properties, or even with just a name, and then use a `Reference` to add information to them.  Here, we'll create a simple reference as (fake) csv file:

In [15]:
import io

# Columns are "Name", "Plate", "Well", "Concentration (nM)", "Sequence"
csv_file = io.StringIO(
    """
Name,Plate,Well,"Concentration (nM)",Sequence
S1,plate1,A2,100000,AGAAT
S2,plate1,A3,125000,GTTCT
S3,plate1,A4,125000,GTTCT
S4,plate1,A5,125000,GTTCT
S5,plate1,B1,125000,GTTCT
S6,plate1,B2,125000,GTTCT
"""
)

ref = Reference.from_csv(csv_file)

Now, we can use `.with_reference` to add information:

In [16]:
Strand("S2")

Strand(name='S2', concentration=<Quantity(NaN, 'nanomolar')>, plate='', well=None, sequence=None)

In [17]:
Strand("S2").with_reference(ref)

Strand(name='S2', concentration=<Quantity(125.000000, 'micromolar')>, plate='plate1', well=WellPos("A3"), sequence='GTTCT')

## Larger mixes

In [18]:
ref = ref

In [28]:
strandmix1 = Mix(
    [
        MultiFixedVolume(
            components=[Strand(f"S{x}") for x in range(1, 4)],
            fixed_volume="2 µL",
            equal_conc="min_volume",
        )
    ],
    "strand mix A",
).with_reference(ref)
strandmix1

Table: Mix: strand mix A, Conc: 38.46 µM, Total Vol: 6.50 µl

| Component   | [Src]     | [Dest]     | #   | Ea Tx Vol   | Tot Tx Vol   | Location           | Note   |
|:------------|:----------|:-----------|:----|:------------|:-------------|:-------------------|:-------|
| S2, S3      | 125.00 µM | 38.46 µM   | 2   | 2.00 µl     | 4.00 µl      | **plate1: A3**, A4 |        |
| S1          | 100.00 µM | 38.46 µM   |     | 2.50 µl     | 2.50 µl      | plate1: A2         |        |
| *Total:*    |           | *38.46 µM* | *3* |             | *6.50 µl*    |                    |        |

In [29]:
strandmix2 = Mix(
    [
        MultiFixedConcentration(
            components=[Strand(f"S{x}") for x in range(4, 7)],
            fixed_concentration="1 µM",
        )
    ],
    "strand mix B",
    fixed_total_volume="100 µL",
).with_reference(ref)
strandmix2

Table: Mix: strand mix B, Conc: 1.00 µM, Total Vol: 100.00 µl

| Component   | [Src]     | [Dest]    | #   | Ea Tx Vol   | Tot Tx Vol   | Location         | Note   |
|:------------|:----------|:----------|:----|:------------|:-------------|:-----------------|:-------|
| S4,         | 125.00 µM | 1.00 µM   | 3   | 800.00 nl   | 2.40 µl      | **plate1: A5**,  |        |
| S5, S6      |           |           |     |             |              | **B1**, B2       |        |
| Buffer      |           |           |     | 97.60 µl    | 97.60 µl     |                  |        |
| *Total:*    |           | *1.00 µM* | *4* |             | *100.00 µl*  |                  |        |

With strands on plates...

In [30]:
sample1 = Mix(
    [
        FixedConcentration(strandmix1, "500 nM"),
        FixedConcentration(strandmix2, "100 nM"),
        FixedConcentration(mg_buffer, "12.5 mM"),
    ],
    name="Sample 1",
    fixed_total_volume="100 µL",
)
sample1

Table: Mix: Sample 1, Conc: 500.00 nM, Total Vol: 100.00 µl

| Component    | [Src]     | [Dest]      | #   | Ea Tx Vol   | Tot Tx Vol   | Location   | Note   |
|:-------------|:----------|:------------|:----|:------------|:-------------|:-----------|:-------|
| strand mix A | 38.46 µM  | 500.00 nM   |     | 1.30 µl     | 1.30 µl      |            |        |
| strand mix B | 1.00 µM   | 100.00 nM   |     | 10.00 µl    | 10.00 µl     |            |        |
| 10× Mg       | 125.00 mM | 12.50 mM    |     | 10.00 µl    | 10.00 µl     |            |        |
| Buffer       |           |             |     | 78.70 µl    | 78.70 µl     |            |        |
| *Total:*     |           | *500.00 nM* | *4* |             | *100.00 µl*  |            |        |

In addition to seeing the series of recipes above...

In [31]:
sample1.all_components()

,concentration_nM,component
name,,
MgCl₂,12500000.0,"Component(name='MgCl₂', concentration=<Quantit..."
S1,500.000000,"Strand(name='S1', concentration=<Quantity(100...."
S2,500.000000,"Strand(name='S2', concentration=<Quantity(125...."
S3,500.000000,"Strand(name='S3', concentration=<Quantity(125...."
S4,100.000000,"Strand(name='S4', concentration=<Quantity(125...."
S5,100.000000,"Strand(name='S5', concentration=<Quantity(125...."
S6,100.000000,"Strand(name='S6', concentration=<Quantity(125...."
